In [2]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax


# Load dataset dari CSV
df = pd.read_csv("datasets/dataset_terakhir.csv")

# Misalnya kolom teks bernama 'text' dan label bernama 'label'
X = df['text'].tolist()
y = df['sentiment'].tolist()

# Split data ke train dan test (contoh: 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function untuk evaluasi model
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f"Performance of {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("-" * 50)

# Evaluate Logistic Regression model
logistic_model_path = "model/logistic_regression_model.pkl"
tfidf_vectorizer_path = "model/tfidf_vectorizer.pkl"

logistic_model = joblib.load(logistic_model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)

X_test_tfidf = tfidf_vectorizer.transform(X_test)
y_pred_logistic = logistic_model.predict(X_test_tfidf)
evaluate_model(y_test, y_pred_logistic, "Logistic Regression")

# Evaluate Random Forest model
random_forest_model_path = "model/random_forest_model.pkl"

random_forest_model = joblib.load(random_forest_model_path)
y_pred_rf = random_forest_model.predict(X_test_tfidf)
evaluate_model(y_test, y_pred_rf, "Random Forest")



from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
from tqdm import tqdm

# Mapping label
label_map = {
    0: "negative",
    1: "neutral",
    2: "positive"
}
reverse_label_map = {v: k for k, v in label_map.items()}
y_test_encoded = [reverse_label_map[label] for label in y_test]

# Load model & tokenizer
transformer_model_dir = "sentinova_sentiment_model_12_epochs_70_30_split"
tokenizer = AutoTokenizer.from_pretrained(transformer_model_dir)
model = AutoModelForSequenceClassification.from_pretrained(transformer_model_dir)
model.eval()

# Jika pakai GPU:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

batch_size = 32
predicted_ids = []

# Batching prediksi
for i in tqdm(range(0, len(X_test), batch_size)):
    batch_texts = X_test[i:i+batch_size]
    inputs = tokenizer(
    batch_texts,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)


    # Jika pakai GPU:
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1).cpu().numpy()
        predicted_ids.extend(preds)

# Evaluasi
evaluate_model(y_test_encoded, predicted_ids, "Transformer-based Model (Batched)")





Performance of Logistic Regression:
Accuracy: 0.8742
Precision: 0.8749
Recall: 0.8742
F1 Score: 0.8741
--------------------------------------------------
Performance of Random Forest:
Accuracy: 0.8664
Precision: 0.8689
Recall: 0.8664
F1 Score: 0.8659
--------------------------------------------------


100%|██████████| 179/179 [00:50<00:00,  3.57it/s]

Performance of Transformer-based Model (Batched):
Accuracy: 0.9164
Precision: 0.9170
Recall: 0.9164
F1 Score: 0.9161
--------------------------------------------------
